In [2]:
from functools import partial
import sys
import pycpd
from matplotlib import pyplot as plt
from pycpd import AffineRegistration
import numpy as np
import time
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import openpyxl
import math

### Main Code
The following code goes through the image stack, working with pairs of two images at a time. Microtubule coordinates from the two images undergo an affine registration and resulting point pairs that are within a threshold distance of each other are determined to be connected (assigned to the same microtubule).

In [ ]:
fileName = "ManualAlignedResults.xlsx" #Excel file holding final microtubule locations

#sort sheets numerically before reading in data
file = pd.ExcelFile(fileName,engine='openpyxl')
sheets = sorted(file.sheet_names)

vertices = [] # coordinates of each vertex (in nm)
connectivity = [] # pairs of vertex numbers that are connected
y = 0. # nm starting axial coordinate
dy = 50. # nm slice thickness
scale = 200./125.75 # nm/pixels
threshold = 11*scale # distance used to determine if nodes in adjacent slices belong to the same MT


#read in data from Excel file
for i, sheet in enumerate(sheets[0:-1]):
    
    #read in coordinates from first image for registration
    if i==0: #for first image in series, read in from spreadsheet
        Xstart = len(vertices) #to keep track of indices in the vertices list
        df = pd.read_excel(fileName, sheet_name=sheet, engine='openpyxl')
        X = df.to_numpy() 
        X = X*scale
        X = np.append(X,np.ones((len(X),1))*y,axis=1)

        for point in X:
            vertices.append(point)

        Xindices = range(Xstart, len(vertices))
    else: 
        X = Y
        Xindices = Yindices
    
    #read in coordinates from second image for registration
    Ystart = len(vertices) #to keep track of indices in the vertices list

    y = y+dy
    df = pd.read_excel(fileName, sheet_name=sheets[i+1], engine='openpyxl')
    Y = df.to_numpy()
    Y = Y*scale
    Y = np.append(Y,np.ones((len(Y),1))*y,axis=1)
    
    for point in Y:
        vertices.append(point)
    
    Yindices = range(Ystart, len(vertices))
    
    #register the points from the two images
    reg = AffineRegistration(tolerance = 0.0001, **{'X': X[:,0:2], 'Y': Y[:,0:2]})
    reg.register()

    #check the proximity of point pairs to determine connectivity
    for index, point in enumerate(X):
        distances = np.linalg.norm(point[0:2]-reg.TY, axis=1)
        if np.min(distances) < threshold:
            connectivity.append((Xindices[index],Yindices[np.argmin(distances)]))

#save coordinate and connectivity data
with open("vertexCoords.txt", "w") as f:
    for v in vertices:
        f.write(str(v) +"\n")
        
with open("connectivityTable.txt", "w") as f:
    for c in connectivity:
        f.write(str(c) +"\n")